# Web3 Python căn bản

## Nội dung
1. Những kiến thức cần tìm hiểu
2. Kết nối Binance Smart Chain public node
3. Thu thập dữ liệu smart contract
4. Thực hành thu thập dữ liệu của Valas <br>
    4.1. Lấy data từ functions của contract <br>
    4.2. Lấy data từ events của contract <br>
    4.3. Thu thập nhiều loại dữ liệu event một lúc

## 1. Những kiến thức cần tìm hiểu trước khi thu thập dữ liệu trên blockchain

- Ethereum Virtual Machine là gì? [link](https://ethereum.org/en/developers/docs/evm/)
- Application Binary Interface là gì? [link](https://www.quicknode.com/guides/solidity/what-is-an-abi) và [link](https://docs.soliditylang.org/en/develop/abi-spec.html#abi-json)
- Light, full and archive ethereum node là gì? [link](https://www.quicknode.com/guides/infrastructure/ethereum-full-node-vs-archive-node) và [link](https://link.springer.com/chapter/10.1007/978-1-4842-3492-1_2)
- Các nguồn tham khảo
    * [Ethereum docs](https://ethereum.org/en/developers/docs/)
    * [Web3 python docs](https://web3py.readthedocs.io/en/stable/)
    * [Ethereum JSON-RPC - Postman](https://documenter.getpostman.com/view/4117254/ethereum-json-rpc/RVu7CT5J)


## 2. Kết nối binance smart chain public node
- Đầu tiên, dữ liệu được lưu trên các blockchain nodes. Ta cần kết nối với các node này để lấy dữ liệu, sử dụng một provide URL
    - Tìm kiếm "rpc node" của các mạng blockchain (eg. ethereum rpc, bsc rpc, ftm rpc, etc.) [link](https://rpc.info/)
    - Tại chuỗi thực hành này, mình sử dụng provider url của binance smart chain ([bsc rpc](https://docs.bscscan.com/misc-tools-and-utilities/public-rpc-nodes))
- Sau đó, ta sử dụng provider URL vừa lấy được để kết nối với một node của BSC chain  
    - Kiểm tra kết nối với node của mainnet
    - Lấy dữ liệu của block mới nhất

In [1]:
from web3 import HTTPProvider
from web3 import Web3
from web3.middleware import geth_poa_middleware
import time
import json

provider_url = "https://bsc-dataseed4.binance.org/" # rpc link
web3 = Web3(HTTPProvider(provider_url))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

#check connect successfull
is_connected = web3.is_connected()
print(f"Successful Connection: {is_connected} ")
#check latest blocknumber
block_number = web3.eth.block_number
print(f"The latest blocknumber is: {block_number}")

Successful Connection: True 
The latest blocknumber is: 36721747


In [2]:
#get latest block transaction data
data = web3.eth.get_block(block_number)
data

AttributeDict({'hash': HexBytes('0xf06148e4d49a5bbbebcc4bbaa0e2b0a40438ccf08b2d88a7b77e4a4216b0d758'),
 'parentHash': HexBytes('0x23331757afb735be8c5d72210877129afea039ad74650f9d2827accd3e16fc60'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'miner': '0xBe807Dddb074639cD9fA61b47676c064fc50D62C',
 'stateRoot': HexBytes('0x401b3e86597415921c0180f565da90d598c7cb1e4f89b7bb08549ec03d294707'),
 'transactionsRoot': HexBytes('0xcf9b21ee797b16bc070de6a822522691ac37f894e2da229afd2a250faecf5ab1'),
 'receiptsRoot': HexBytes('0x07aead7d340ed286f2c4a0a66aff5ffff460f22cbbb479d3e986b4ded21b5a81'),
 'logsBloom': HexBytes('0x6834a745af006118e01c40789681f6c0844702a4930728c51ce46d76120825d6b38bf32c14269900e8aa16af2827ab8a087c5840473a617892ce188f43a6b2c03e30ac9063d02c69c1c2050b8937203122342b248d570d6028655096b4ec64c8d4de0e68d35f03a40a450857046ca9408d583b91416f8c3180c4e3db40ca234b6990fd21087b58930e91a2488d4178d1e27cd4c3a8627908f3420256e14013bd4682fa0238c46

## 3. Thu thập dữ liệu của smart contract
*Thực hành thu thập dữ liệu của lending smart contract Valas trên Binance smart chain. Để có thể lấy dữ liệu của Valas ta cần có:*
- *Địa chỉ smart contract*
- *ABI của Valas*

*Địa chỉ smart contract không được Valas công bố nhưng bằng những nghiệp vụ chuyên dụng mình đã thăm dò và lấy được địa chỉ này trên [BSCscan](https://bscscan.com/). Dưới đây ta sẽ tiến hành lấy ABI của Valas*

### Cách lấy ABI
*Đối với các smart contract cho phép lấy ABI trên BSCscan, ta có thể  thực hiện các bước sau để lấy ABI:*  
1. Truy cập [BSC Scan](https://bscscan.com/).
2. Nhập địa chỉ smart contract address vào search bar.
3. Kéo xuống và chon tab “Contract”.

![image.png](../images/contract.png)

4. Kéo xuống thêm một tý, chúng ta sẽ thấy phần ABI.
5. Chọn nút copy ABI  hoặc download dưới dạng json.

![image.png](../images/ABI.png)

*Lưu ý:*

> Một số smart contract không công bố ABI, ta có thể tìm kiếm source code của nó trên github, deploy và lấy ABI.

> Đối với các smart contract không công bố cả code cũng như ABI như Valas, mình có thể đọc [docs](https://docs.valasfinance.com/) của nó. Tại [đây](https://docs.valasfinance.com/#why-valas), mình thấy rằng Valas sử dụng công nghệ của Aave Finance. Do Aave finance public [ABI](https://docs.aave.com/developers/v/2.0/deployed-contracts/deployed-contracts) nên mình có thể sử dụng ABI này. Mặc dù một số điểm trên ABI của Aave có thể đã bị Valas thay đổi, nhưng nhìn chung là có thể dùng tạm để thay thế cho ABI của Valas.

## 4. Thực hành thu thập dữ liệu của Valas
*Tại đây, mình sẽ demo một số cách lấy dữ liệu của một smart contract. Chúng ta có hai loại dữ liệu cần lấy, đó là:*
- *Dữ liệu từ các giao dịch (transaction) của smart contract*
- *Dữ liệu trạng thái là các thông tin từ các phần tử liên quan với smart contract (với lending pool Valas, các phần tử  này là các tokens và users).*

*Việc thu thập dữ liệu sẽ dựa trên ABI của smart contract.*

In [3]:
#lấy cấu trúc dữ liệu của abi
with open("../abi/lending_pool_aave_v2.json", "r") as f:
    abi = json.loads(f.read())
#Địa chỉ của valas  
address = "0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5"
# Kiểm tra xem địa chỉ có đúng không
if not web3.is_address(address):
    address = web3.to_checksum_address(address)
#Sử dụng web3 thiết lập đối tượng contract để sử dụng các phương thức
contract = web3.eth.contract(abi=abi, address=address)

> **NOTE**: Nên kiểm tra xem chuỗi byte code có phải là một địa chỉ trên blockchain hay không: **`web3.isAddress(address)`**. Nếu là địa chỉ, nên có thêm một bước convert địa chỉ thành địa chỉ thực trên web3 bằng **`web3.toChecksumAddress(address)`**. Do địa chỉ trên của mình đã là địa chỉ thực của Valas trên BSC nên hai bước này có thể bỏ qua.

### 4.1. Lấy data từ **function** của contract 
*Mình sử dụng một số hàm trong smart contract để lấy dữ liệu từ các tokens được giao dịch trong lending pool Valas. Đầu tiên sẽ lấy danh sách địa chỉ smart contract của các tokens được giao dịch. Sau đó, lấy thông tin của tokens dựa trên địa chỉ của nó. Thông tin của user được lấy tương tự như vậy (Không có hàm lấy danh sách users, vì số lượng users rất nhiều)*

> **Note**: Cấu trúc các hàm này trong ABI thường sẽ có trường **`"stateMutability": "view"`** hoặc có chữ **`get`** đầu tiên của tên hàm

#### 4.1.a. Lấy địa chỉ các token
*Ta có thể tìm các hàm hỗ trợ lấy dữ liệu trong file ABI. Tại đây, mình đã sử dụng hàm getReservesList trong `abi/lending_pool_aave_v2.json`. Các bạn có thể thấy cấu trúc của nó ở hình dưới hoặc vào file ABI này tìm kiếm thử.*

![image.png](../images/list_token.png)

In [4]:
contract.functions.getReservesList().call()

['0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c',
 '0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56',
 '0x8AC76a51cc950d9822D68b83fE1Ad97B32Cd580d',
 '0x55d398326f99059fF775485246999027B3197955',
 '0x1AF3F329e8BE154074D8769D1FFa4eE058B1DBc3',
 '0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82',
 '0x2170Ed0880ac9A755fd29B2688956BD959F933F8',
 '0x7130d2A12B9BCbFAe4f2634d864A1Ee1Ce3Ead9c',
 '0x14016E85a25aeb13065688cAFB43044C2ef86784',
 '0x40af3827F39D0EAcBF4A168f8D4ee67c121D11c9']

>**Note**: Hàm trên mặc định lấy dữ liệu ở block mới nhất. Ngoài ra ta có thể dữ liệu của các block trong quá khứ thông qua parameter *`block_identifier`* của hàm `call()`, các bạn có thể tham khảo tại [document](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=call#web3.contract.ContractFunction.call)

#### 4.1.b. Lấy thông tin của một token trong Valas
*Để lấy thông tin của một token trong danh sách token được reserved trong Valas trên, mình sử dụng hàm **`getReserveData()`**. Tại đây mình cũng sẽ thử lấy dữ liệu trong block ở quá khứ (cụ thể là block cách block mới nhất 10 block), sử dụng tham số `block_identifier` của hàm `call`.*

> Lưu ý là ta đang sử dụng full node provider. Full node chỉ lưu dữ liệu state của blockchain tại 128 blocks gần nhất (Đọc tìm hiểu full, archive và light node ở đầu series này đi nhé, quan trọng đấy). Vì vậy ta sẽ không thể lấy dữ liệu ở những block cách block mới nhất quá 127 block

In [5]:
blocknumber = web3.eth.block_number - 10
contract.functions.getReserveData("0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c").call(block_identifier=blocknumber)

((92234085768165225799488,),
 1115805936301057095376817033,
 1442820529145724717421810036,
 62278538552699761212585500,
 233151161917470347430923573,
 10000000000000000000000000,
 1709690813,
 '0xB11A912CD93DcffA8b609b4C021E89723ceb7FE8',
 '0x2Adc0c94A055f1FF64A35672D30Eb523ec647816',
 '0xE7CDC4e53915D50B74496847EeBa7233caE85CE5',
 '0xe4630EaE1C2b8B30F9b4AADD355522Ec9c18bfe4',
 0)

### 4.2. Lấy data từ **event** của contract 
*Tiếp theo mình sẽ hướng dẫn các bạn crawl dữ liệu từ các giao dịch, thông qua **event** mà các contract emit ra.*
*Không như dữ liệu về state của blockchain chỉ được lưu của 128 block đầu, full node lưu tất cả lịch sử giao dịch của smart contract trên BSC. Tại đây, mình sẽ crawl dữ liệu của deposit event được emit từ các giao dịch của Valas tại block 21031205. Hình dưới đây là cấu trúc của event này trong ABI. Các bạn có thể kiểm tra trong `abi/lending_pool_aave_v2.json`*

![image.png](../images/deposit.png)

In [6]:
events = contract.events.Deposit.create_filter(fromBlock=21031205, toBlock=21031205).get_all_entries()
event_list = []
for event in events:
    event_list.append(json.loads(web3.to_json(event)))
event_list

[{'args': {'reserve': '0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82',
   'onBehalfOf': '0x26417D19dc19a73274B4c97615b310b39017FA57',
   'referral': 0,
   'user': '0x26417D19dc19a73274B4c97615b310b39017FA57',
   'amount': 1257749112692222427},
  'event': 'Deposit',
  'logIndex': 220,
  'transactionIndex': 71,
  'transactionHash': '0x0ee0f8ee33047ba977e3afd0b0995885b8d304b1aa1f1c6d307099c9ef9206e9',
  'address': '0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5',
  'blockHash': '0x68dee16ed601595adeb352f2072879d04f288506e5dcdd44f064b4f862e6bfe4',
  'blockNumber': 21031205}]

>**NOTE**: Các bạn có thấy hai trường **fromBlock** và **toBlock** không? Mình có thể thay đổi hai giá trị này để  thu thập dữ liệu trong nhiều blocks nhé. Chú ý là **fromBlock** luôn nhỏ hơn **toBlock** và số  blocks tối đa có thể crawl một lần là **5000** blocks thôi. Ngoài ra có thể tham khảo trên [Web3 python docs](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=createFilter#web3.contract.Contract.events.your_event_name.createFilter) để thiết lập các điều kiện lọc dữ liệu event khác theo ý thích nhé.

### 4.3. Thu thập nhiều loại dữ liệu event cùng lúc
*Với cách trên, mọi người có thể thấy nhược điểm là mỗi lần mình chỉ có thể thu thập được một loại dữ liệu thôi. Vậy làm sao để thu thập nhiều loại dữ liệu một lúc để tránh mỗi lần thu thập lại phải quét lại những blocks đã crawl? Ta sẽ phải tự code.* 

*Cơ bản ta sẽ sử dụng event hash để crawl. Event hash là một chuỗi byte code được mã hóa theo chuẩn KECCAK_256 từ một chuỗi định nghĩa hàm tạo bởi cấu trúc event trong ABI (Đọc code để hiểu rõ hơn nhé - hàm **`get_topic_filter`** trong `service/utils.py`).*

*Tại đây, mình cũng sẽ sẽ tách hai event là `deposit` và `borrow` trong ABI của aave tạo thành một ABI mới riêng cho hai event này tại `abi/event_abi.json`.*

*Mình có xây dựng một class EthReceiptLogHandler cung cấp các functions xử lý ABI và dữ liệu event trả về. Có ba functions quan trọng mà mọi người cần chú ý đó là:* 

- *(1) **`build_list_info_event`** xây dựng thông tin event từ ABI. Hàm này có nhiệm vụ trả về `EventSubscriber` chứa thông tin đối sánh tên event theo event hash và thứ tự các trường dữ liệu tương ứng với dữ liệu event mà web3 trả về.*

- *(2) **`web3_dict_to_receipt_log`** biến đổi dữ liệu event data trả về theo cấu trúc mà mình quy định sẵn (cài này tạo ra để mình xử lý event dễ hơn ý mà).*

- *(3) **`extract_event_from_log`** đối sánh dữ liệu event trả về với `EventSubscriber` và giải mã các trường dữ liệu `topics` và `data` trong event (Muốn biến hai trường này là gì thì các bạn đọc phần tài liệu về ABI mình để ở đầu bài nhé).*

- *(4) cuối cùng **`eth_event_to_dict`** chuyển đổi dữ liệu event về cấu trúc event cuối cùng (theo dictionary) lưu vào cơ sở dữ liệu (Cái này tạo ra để mình có thể lưu dữ liệu vào mongodb).*

In [7]:
from receipt_log_handler import EthReceiptLogHandler

with open("../abi/event_abi.json", "r") as f:
    event_abi = json.loads(f.read())

handler = EthReceiptLogHandler()
# xây dựng thông tin event từ ABI.
event_abi_info = handler.build_list_info_event(event_abi)
event_abi_info

[[<model.receipt_log.EventSubscriber at 0x109b00490>,
  '0xc6a898309e823ee50bac64e45ca8adba6690e99e7841c45d754e2a38e9019d9b',
  ['reserve', 'user', 'onBehalfOf'],
  'Borrow'],
  '0xde6857219544bb5b7746f48ed30be6386fefc61b2f864cacf559893bf50fd951',
  ['reserve', 'user', 'onBehalfOf'],
  'Deposit']]

In [8]:
# tách lấy event hash
event_hash = [event_info[1] for event_info in event_abi_info]
event_hash

['0xc6a898309e823ee50bac64e45ca8adba6690e99e7841c45d754e2a38e9019d9b',
 '0xde6857219544bb5b7746f48ed30be6386fefc61b2f864cacf559893bf50fd951']

In [9]:
# tách lấy event subscriber
event_subscriber = {}
for info in event_abi_info:
    event_subscriber[info[1]] = info[0]

event_subscriber

{'0xc6a898309e823ee50bac64e45ca8adba6690e99e7841c45d754e2a38e9019d9b': <model.receipt_log.EventSubscriber at 0x109b00490>,
 '0xde6857219544bb5b7746f48ed30be6386fefc61b2f864cacf559893bf50fd951': <model.receipt_log.EventSubscriber at 0x109b00400>}

*Do dữ liệu event trên blockchain vô cùng lớn và nhiểu dữ liệu từ các lending pool khác nhau, mình đã tạo một filter chỉ thu thập dữ liệu của Valas. Filter này gồm 4 điều kiện là fromBlock, toBlock, topics - danh sách event hash cần lọc (event hash lưu ở topics[0] nên mình để mảng event hash làm phần tử đầu của topics - hơi khó hiểu nhỉ, mọi người sẽ hình dung rõ hơn khi nhìn vào dữ liệu thô) và address là các địa chỉ smart contract mà mình muốn lấy event.*

In [12]:
#create filter
filter_params = {
    "fromBlock":21043181,
    "toBlock":21043646,
    "topics": [event_hash],
    "address":["0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5"]
}

# event_filter = web3.eth.filter(filter_params)
# events = event_filter.get_all_entries()
# event_list = []
# for event in events:
#     log = handler.web3_dict_to_receipt_log(event)
#     eth_event = handler.extract_event_from_log(log, event_subscriber[log.topics[0]])
#     if eth_event is not None:
#         eth_event_dict = handler.eth_event_to_dict(eth_event)
#         event_list.append(eth_event_dict)

event_filter = web3.eth.filter(filter_params)
event_logs = event_filter.get_all_entries()
event_list = []
for event_log in event_logs:
    log = handler.web3_dict_to_receipt_log(event_log)
    eth_event = handler.extract_event_from_log(log, event_subscriber[log.topics[0]])
    if eth_event is not None:
        eth_event_dict = handler.eth_event_to_dict(eth_event)
        event_list.append(eth_event_dict)


web3.eth.uninstall_filter(event_filter.filter_id)

True

*Dưới đây là dữ liệu thô, trường `topics` là một mảng các phần tử nhé. Phần tử đầu tiên chính là event hash. Các phần tử còn lại sẽ là các trường dữ liệu có `index` là `true` trong cấu trúc event ở ABI.*

In [13]:
event_logs

[AttributeDict({'address': '0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5',
  'topics': [HexBytes('0xde6857219544bb5b7746f48ed30be6386fefc61b2f864cacf559893bf50fd951'),
   HexBytes('0x0000000000000000000000000e09fabb73bd3ade0a17ecc321fd13a19e81ce82'),
   HexBytes('0x0000000000000000000000007c6defb490b8ffe9379839914b4073c35ea7468a'),
   HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')],
  'data': HexBytes('0x0000000000000000000000007c6defb490b8ffe9379839914b4073c35ea7468a0000000000000000000000000000000000000000000000058282d58b806d99f4'),
  'blockNumber': 21043181,
  'transactionHash': HexBytes('0x4452f763a53dad9fb3a9784a2133a4a960379685d194932155fa6a7217980ad9'),
  'transactionIndex': 149,
  'blockHash': HexBytes('0x52ac94f0072e20b23c7bab220d8c9e3d43185cdbc62bfec962076a2de4a51912'),
  'logIndex': 409,
  'removed': False}),
 AttributeDict({'address': '0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5',
  'topics': [HexBytes('0xde6857219544bb5b7746f48ed30be6386fefc61b2

*Đây là dữ liệu sau cuối cùng trả về . Quá trình giải mã dữ liệu khá phức tạp nhưng mình sẽ giải thích đơn giản như sau:*

*Dựa vào cấu trúc event trong ABI, mình phân loại các trường dữ liệu theo hai loại có thuộc tính **`index`** là `true` và `false`:*
- *Với những trường dữ liệu có **`index`** là `true`, theo thứ tự của chúng trong event ABI sẽ tương ứng là các giá trị theo thứ tự trong **`topics`** (trừ `topics[0]` là event hash nhé).* 
- *Các trường dữ liệu có **index** là `false` theo thứ tự tương ứng với mỗi 32 bytes trong **`data`***

*Sau khi đã xác định vị trí các trường dữ liệu, mình dựa vào thuộc tính **`type`** của chúng trong ABI để giải mã:*
- *Nếu **`type`** là `address` mình sẽ lấy 20 bytes cuối rồi ghép với `0x`*
- *Còn **`type`** là `uint` mình sẽ quy đổi giá trị từ thập lục phân sang thập phân.*

(Bạn nào chưa hiểu thì đọc code mình để dễ hiểu hơn nhé, hoặc đọc [document về Event Log Filter](https://web3py.readthedocs.io/en/stable/filters.html?highlight=get_transaction#event-log-filters)).

*Dưới đây là dữ liệu cuối cùng thu được.*

In [14]:
event_list

[{'type': 'event',
  'event_type': 'DEPOSIT',
  'contract_address': '0xe29a55a6aeff5c8b1beede5bcf2f0cb3af8f91f5',
  'transaction_hash': '0x4452f763a53dad9fb3a9784a2133a4a960379685d194932155fa6a7217980ad9',
  'log_index': 409,
  'block_number': 21043181,
  'reserve': '0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82',
  'onBehalfOf': '0x7c6defb490b8ffe9379839914b4073c35ea7468a',
  'referral': '0',
  'user': '0x7c6defb490b8ffe9379839914b4073c35ea7468a',
  'amount': '101638034135582611956'},
 {'type': 'event',
  'event_type': 'DEPOSIT',
  'contract_address': '0xe29a55a6aeff5c8b1beede5bcf2f0cb3af8f91f5',
  'transaction_hash': '0xbddb09a7f95e47bcf469b26f621cb62fa050cf1f4f0cb9678352acf815a15e89',
  'log_index': 169,
  'block_number': 21043427,
  'reserve': '0xe9e7cea3dedca5984780bafc599bd69add087d56',
  'onBehalfOf': '0x19ec9e3f7b21dd27598e7ad5aae7dc0db00a806d',
  'referral': '0',
  'user': '0x19ec9e3f7b21dd27598e7ad5aae7dc0db00a806d',
  'amount': '256058331951835542063'},
 {'type': 'event',
  'ev

#### Thu thập transaction sinh ra deposit event
*Mọi người có thể thấy trong dữ liệu block và dữ liệu event đều có một trường `transaction_hash`. Dựa vào nó ta có thể thu thập dữ liệu transaction mình mong muốn. Dưới đây là dữ liệu từ transaction hash của deposit event của block 21031205.*

In [15]:
web3.eth.get_transaction("0x0ee0f8ee33047ba977e3afd0b0995885b8d304b1aa1f1c6d307099c9ef9206e9")

AttributeDict({'blockHash': HexBytes('0x68dee16ed601595adeb352f2072879d04f288506e5dcdd44f064b4f862e6bfe4'),
 'blockNumber': 21031205,
 'from': '0x26417D19dc19a73274B4c97615b310b39017FA57',
 'gas': 752442,
 'gasPrice': 5000000000,
 'hash': HexBytes('0x0ee0f8ee33047ba977e3afd0b0995885b8d304b1aa1f1c6d307099c9ef9206e9'),
 'input': HexBytes('0xe8eda9df0000000000000000000000000e09fabb73bd3ade0a17ecc321fd13a19e81ce8200000000000000000000000000000000000000000000000011746c27769529db00000000000000000000000026417d19dc19a73274b4c97615b310b39017fa570000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': 476,
 'to': '0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5',
 'transactionIndex': 71,
 'value': 0,
 'type': 0,
 'chainId': 56,
 'v': 148,
 'r': HexBytes('0xedf56947061ff71ad62dfded9fb7ba19fc271d6f52d8df2a207d405ca473da87'),
 's': HexBytes('0x34ce34ec26f2a48cc85ba8a759333dd59b6d106597c32039ddc2e9aa942773c8')})

> Note: Đến đây mọi người có thể thắc mắc là thế với một list các transaction hash thì sao? Rất tiếc hiện tại mình chưa thấy hàm nào của web3 py hỗ trợ hoặc có thể do mình tìm chưa kỹ. Tuy nhiên ta có thể cải tiến quá trình bằng cách lấy dữ liệu theo lô (batch) với đa luồng (multi-processing). Kiến thức này hơi nâng cao, mình sẽ hướng dẫn ở một bài thực hành khác phức tạp hơn.

## Xây dựng thu thập dữ liệu một luồng liên tục
*Phần này bonus cho mọi người. Mọi người có thể đọc tiếp hoặc bỏ qua. Nói chung, nó chỉ tổng hợp lại những kiến thức ở trên để xây dựng một con crawler thu thập dữ liệu event liên tục.*

In [38]:
from receipt_log_handler import EthReceiptLogHandler

def craw_events(
    web3, 
    from_block=0, 
    to_block=None, 
    event_abi_file="../abi/event_abi.json", 
    contract_addresses=["0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5"],
    block_size=100,
    stop = False
):
    latest_block = web3.eth.block_number
    if to_block and to_block > latest_block:
        to_block=latest_block
        print(f"To_block > latest_block, set to_block = {latest_block}")
    with open(event_abi_file, "r") as f:
        event_abi = json.loads(f.read())

    handler = EthReceiptLogHandler()
    # xây dựng thông tin event từ ABI.
    event_abi_info = handler.build_list_info_event(event_abi)
    # lấy event hash
    event_hash = [event_info[1] for event_info in event_abi_info]
    # tách lấy event subscriber
    event_subscriber = {}
    for info in event_abi_info:
        event_subscriber[info[1]] = info[0]
    
    if not to_block:
        to_block = web3.eth.block_number
    
    _from = from_block
    _to = from_block + block_size
    
    while True:
        if _to > to_block:
            _to = to_block
                
        print(f"Start crawling data from {_from} to {_to}")
        #create filter
        filter_params = {
            "fromBlock":_from,
            "toBlock":_to,
            "topics": [event_hash],
            "address":contract_addresses
        }

        event_filter = web3.eth.filter(filter_params)
        try:
            events = event_filter.get_all_entries()
            event_list = []
            for event in events:
                log = handler.web3_dict_to_receipt_log(event)
                eth_event = handler.extract_event_from_log(log, event_subscriber[log.topics[0]])
                if eth_event is not None:
                    eth_event_dict = handler.eth_event_to_dict(eth_event)
                    print("Event data:\n", eth_event_dict)
    
            web3.eth.uninstall_filter(event_filter.filter_id)
            if _to == to_block:
                if stop:
                    print("Stop crawling data...")
                    break
                else:
                    to_block = web3.eth.block_number
                    print(f"Get latest block {to_block}")
        except Exception as e:
            if str(e) == "{'code': -32005, 'message': 'limit exceeded'}":
                print(f"Get err: {str(e)}! Sleep for 5 seconds...!")
                time.sleep(5)
                pass
            else:
                raise e
        
        _from = _to
        _to += block_size

*Mọi người có thể đặt biến `stop = False` để cho chạy liên tục. Có thể sửa hàm để xuất dữ liệu ra file*

In [39]:
craw_events(
    web3, 
    from_block=21042695, 
    to_block=21043695,
    event_abi_file="../abi/event_abi.json", 
    contract_addresses=["0xE29A55A6AEFf5C8B1beedE5bCF2F0Cb3AF8F91f5"],
    block_size=100,
    stop = True
)

Start crawling data from 21042695 to 21042795
Start crawling data from 21042795 to 21042895
Get err: {'code': -32005, 'message': 'limit exceeded'}! Sleep for 5 seconds...!
Start crawling data from 21042895 to 21042995
Event data:
 {'type': 'event', 'event_type': 'DEPOSIT', 'contract_address': '0xe29a55a6aeff5c8b1beede5bcf2f0cb3af8f91f5', 'transaction_hash': '0x9d7f541831a499d00815147057d593cb4894cb3c74cd1c267714378dd808fe96', 'log_index': 148, 'block_number': 21042985, 'reserve': '0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d', 'onBehalfOf': '0x19ec9e3f7b21dd27598e7ad5aae7dc0db00a806d', 'referral': '0', 'user': '0x19ec9e3f7b21dd27598e7ad5aae7dc0db00a806d', 'amount': '1736000000000000000000'}
Start crawling data from 21042995 to 21043095
Start crawling data from 21043095 to 21043195
Get err: {'code': -32005, 'message': 'limit exceeded'}! Sleep for 5 seconds...!
Start crawling data from 21043195 to 21043295
Start crawling data from 21043295 to 21043395
Get err: {'code': -32005, 'message': '

# BTVN

Chú ý: Tất cả dữ liệu crawl được lưu vào file json, theo format trong thư mục ```btvn_examples```

## Chung
1. Tính số transaction trung bình mỗi block của 100 blocks mới nhất.
2. Cho địa chỉ token `0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402`, dựa vào địa chỉ token và ERC_20 abi trong `abi/erc_20.json` thực hiện các nhiệm vụ sau:<br>
    a. Thu thập Transfer event của token đó trong 1000 blocks gần nhất. <br>
    b. Tìm decimals của token.<br>
    c. Tính lượng cung (supply) của token đó. <br>
    d. Tính số dư (balance) token mới nhất của một địa chỉ ví thực hiện nhiều Transfer event nhất trong tập dữ liệu Transfer event đã crawl (gợi ý: địa chỉ này nằm trong trường `_from` của event). <br>
    e. Tính số dư token mới nhất trong địa chỉ ví là địa chỉ ví nhận trong nhiều Transfer event nhất trong tập dữ liệu Transfer event đã crawl (gợi ý: địa chỉ này nằm trong trường `_to` của event). <br>
    f. Tìm ký hiệu (symbol) và tên (name) của token.

Chú ý: Ngoài thông tin trả lời, cần ghi rõ lại start - end block mà mình thu thập dữ liệu, địa chỉ token và ví thu thập. Các giá trị lượng cung và số dư phải chia cho decimals của token.

## AAVE Team
Làm lại các bước trong bài thực hành trên với [Radiant protocol](https://docs.radiant.capital/radiant/contracts-and-security/bnb-chain-contracts) là một lending pool tương tự như Valas trên mạng BSC. Ngoài ra, thực hiện thêm một số bước sau. (Đọc chú ý phía dưới trước khi làm)
1. Crawl dữ liệu event theo 5 loại: Deposit, Borrow, Withdraw, Repay và Liquidate trong 100.000 blocks gần nhất.
2. Crawl dữ liệu transaction của những event đã crawl. 
3. Lấy thông tin địa chỉ token được thực hiện giao dịch Deposit và Borrow nhiều nhất (gợi ý: địa chỉ token nằm trong trường `reserve` trong event, nếu số event của các token bằng nhau thì lấy token bất kỳ)
4. Lấy thông tin địa chỉ thực hiện transaction nhiều nhất trong lending pool (gợi ý: địa chỉ ví nằm trong trường `from` của dữ liệu transaction, nếu số transaction của các địa chỉ bằng nhau thì lấy địa chỉ bất kỳ).
5. Dựa vào địa chỉ Oracle ([Aave oracle](https://docs.radiant.capital/radiant/contracts-and-security/bnb-chain-contracts)) và `abi/oracle_abi.json` tìm giá (asset price) của token ở câu 3.

Chú ý: Các hàm lấy dữ liệu đã có trong `abi/lending_pool_aave_v2.json`. Dựa vào cấu trúc dữ liệu trả về trong abi, chỉ ra ý nghĩa của từng trường dữ liệu crawl về trong câu 3,4.

## Uniswap Team
Dựa vào masterchef V2 [address](https://docs.pancakeswap.finance/code/smart-contracts/main-staking-masterchef-contract), `abi/masterchef_abi.json` và pid = 3 của pancakeswap (tương tự như uniswap) trên bsc. Thực hiện các nhiệm vụ sau. (Đọc chú ý trước khi làm)

1. Lâý thông tin của lp_token từ id (gợi ý: một lp_token cũng tương tự như một pool, vậy nên lấy thông tin lp_token có thể thay bằng lấy thông tin của pool).
2. Tìm địa chỉ lp_token.
3. Dựa vào `abi/lp_token_abi.json` và địa chỉ lp_token tìm được lấy địa chỉ cặp swap và thông tin về reserves của cặp swap, từ địa chỉ lp_token và abi `abi/lp_token_abi.json`.
4. Crawl dữ liệu event Swap, Approval, Burn, Mint và Transfer của lp_token trong 1000 blocks gần nhất với ABI `abi/lp_token_abi.json`.
5. Crawl dữ liệu transaction tương ứng với dữ liệu events.
6. Tìm user giao dịch nhiều nhất (gợi ý: địa chỉ user là trường `from` trong transaction)
7. Tìm thông tin của user ở câu 6 dựa trên địa chỉ masterchef và abi `abi/masterchef_abi.json`.

Chú ý: Dựa vào cấu trúc dữ liệu trả về trong abi, chỉ ra ý nghĩa của từng trường dữ liệu crawl về trong câu 1,7.